In [ ]:
import cv2
import numpy as np
import os

crop_size = 512
stride = 256
threshold_pixels = 512 * 512 * 0.05  # Minimum number of non-zero pixels required

image = cv2.imread('../data/original/jcsmr.jpg')
grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

canny_image = cv2.Canny(grayscale_image, 100, 200)
harris_image = cv2.cornerHarris(np.float32(grayscale_image), blockSize=2, ksize=3, k=0.04)
harris_dilated = cv2.dilate(harris_image, None)
harris_binary = (harris_dilated > 0.01 * harris_dilated.max()).astype(np.uint8) * 255

def save_crops(image, canny_img, harris_img, prefix="crop"):
    h, w = image.shape[:2]
    count = 0
    skipped = 0

    for y in range(0, h - crop_size + 1, stride):
        for x in range(0, w - crop_size + 1, stride):
            rgb_crop = image[y:y + crop_size, x:x + crop_size]
            canny_crop = canny_img[y:y + crop_size, x:x + crop_size]
            harris_crop = harris_img[y:y + crop_size, x:x + crop_size]

            if np.count_nonzero(canny_crop) < threshold_pixels and np.count_nonzero(harris_crop) < threshold_pixels:
                skipped += 1
                continue

            cv2.imwrite(f"../data/filtered_crops/rgb/{prefix}_{count}.png", rgb_crop)
            cv2.imwrite(f"../data/filtered_crops/canny/{prefix}_{count}.png", canny_crop)
            cv2.imwrite(f"../data/filtered_crops/harris/{prefix}_{count}.png", harris_crop)

            count += 1

    print(f"Saved {count} crops with structure.")
    print(f"Skipped {skipped} low-information crops.")

os.makedirs("../data/filtered_crops/rgb", exist_ok = True)
os.makedirs("../data/filtered_crops/canny", exist_ok = True)
os.makedirs("../data/filtered_crops/harris", exist_ok = True)

save_crops(image, canny_image, harris_binary)